# Д/З 2: Извлечение коллокаций + NER
### Выполнила Елизавета Клыкова, БКЛ181
### Пункт 0: получение и очистка датасета
*Выберите корпус отзывов на товары одной из категорий Amazon: http://jmcauley.ucsd.edu/data/amazon/.*

Я остановилась на датасете [5-core Kindle Store](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz) -- архив с ним весит 259 Мб, поэтому на гитхаб не выкладываю. Перед запуском программы его нужно распаковать.

Заранее извиняюсь за некоторую костыльность кода, заключающуюся в бесконечном создании / открывании файлов, но на обработку всего каждый раз мне не хватало ни оперативной памяти, ни терпения :(

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import os
import re
import ast
import json
import html
import random
import pickle
import numpy as np
import pandas as pd
from heapq import nlargest
from tqdm.auto import tqdm
from collections import Counter

import gensim
from gensim.models import KeyedVectors

from summa import keywords

import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.collocations import *

In [3]:
seed = 117
random.seed(seed)
np.random.seed(seed)

(можно не запускать, все сохранено в файл)

In [4]:
# не забыть распаковать архив
with open('Kindle_Store_5.json', encoding='utf-8') as f:
    rev_lines = f.readlines()

In [5]:
reviews = [json.loads(rev) for rev in tqdm(rev_lines)]
item_ids = set([rev['asin'] for rev in reviews])

  0%|          | 0/982619 [00:00<?, ?it/s]

Теперь возьмем [метаданные](http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Kindle_Store.json.gz) (архив весит 96 Мб).

In [6]:
# не забыть распаковать архив
with open('meta_Kindle_Store.json', encoding='utf-8') as f:
    meta_lines = f.readlines()

In [7]:
meta = []
for line in tqdm(meta_lines):
    item_meta = json.loads(line)
    # сразу берем только нужные метаданные
    if item_meta['asin'] in item_ids:
        meta.append(item_meta)

  0%|          | 0/491670 [00:00<?, ?it/s]

In [8]:
meta_ids = set([product['asin'] for product in meta])

In [9]:
meta_dict = {item['asin']: item for item in tqdm(meta)}

  0%|          | 0/31889 [00:00<?, ?it/s]

In [10]:
# так и не смогла побороть df.merge(), так что будет костыль
reviews_with_meta = []

for review in tqdm(reviews):
    prod_id = review['asin']
    if prod_id in meta_ids:
        meta_info = meta_dict[prod_id]
        review_info = {'asin': prod_id,
                       'reviewerID': review['reviewerID'],
                       'overall': review['overall'],
                       'summary': review['summary'],
                       'reviewText': review['reviewText'],
                       'title': meta_info['title'],
                       'brand': meta_info['brand'],
                       'main_cat': meta_info['main_cat'],
                       'category': meta_info['category'],
                       'description': meta_info['description'],
                       'feature': meta_info['feature']}
        reviews_with_meta.append(review_info)

  0%|          | 0/982619 [00:00<?, ?it/s]

In [11]:
rev_df = pd.DataFrame(reviews_with_meta)
rev_df.astype(str).drop_duplicates(inplace=True)
rev_df.reset_index(drop=True, inplace=True)
len(rev_df)  # 485925

485925

In [12]:
len([t for t in rev_df['title'].tolist() if t])  # 485134

485134

In [13]:
len([t for t in rev_df['description'].tolist() if t])  # 0

0

In [14]:
len([t for t in rev_df['feature'].tolist() if t])  # 0

0

Название товара почти у всех отзывов на месте, а вот колонки description и feature оказались пустыми, избавимся от них.

In [15]:
rev_df.drop(columns=['description', 'feature'], inplace=True)

Немного почистим метаданные (приведем в более читаемый формат).

In [16]:
def clean_string(some_string):
    no_html = html.unescape(some_string)
    no_tags = re.sub('<.*?>', '', no_html)
    no_abrs = no_tags.replace(' w/', ' with ').replace('(w/', '(with ')
    return no_abrs

In [17]:
rev_df['title'] = rev_df['title'].apply(lambda x: clean_string(x))

In [18]:
def clean_column(df, some_column):
    values = [[clean_string(value) for value in row]
              for row in df[some_column].tolist()]
    df[some_column] = [[v for v in row if v] for row in values]

In [19]:
clean_column(rev_df, 'category')

Также удалим строки, в которых нет текста отзыва, названия товара или категорий.

In [20]:
rev_df.replace('', np.nan, inplace=True)
rev_df['category'] = rev_df['category'].apply(
    lambda x: np.nan if len(x) == 0 else x)
rev_df.dropna(subset=['reviewText', 'title', 'category'], inplace=True)
len(rev_df)  # 485125

485125

In [21]:
rev_df.to_csv('kindle_reviews.tsv', sep='\t', index=False)

Поскольку файлы очень тяжелые, а у меня маниакальная любовь перезапускать тетрадки, сохраним полученный датафрейм в файл, чтобы можно было считывать оттуда и не повторять препроцессинг каждый раз.

In [22]:
rev_df = pd.read_csv('kindle_reviews.tsv', sep='\t')

In [23]:
def string_to_list(df, column):
    str_reprs = df[column].tolist()
    df[column] = [ast.literal_eval(s) for s in str_reprs]

In [24]:
string_to_list(rev_df, 'category')

In [25]:
rev_df.head()

,asin,reviewerID,overall,summary,reviewText,title,brand,main_cat,category
0,B0012W11D0,AL6WDE46RJ9I3,4.0,HOT TICKET by K. A. Mitchell,I loved the wonderfulDiving in Deepby K. A. Mi...,Hot Ticket (Serving Love) - Kindle edition,Visit Amazon's K.A. Mitchell Page,Buy a Kindle,"[Kindle Store, Kindle eBooks, Literature & Fic..."
1,B0012W11D0,A14R9XMZVJ6INB,3.0,"hot while you are reading it, easily forgotte...","Light, short novella, nothing too deep - Cade ...",Hot Ticket (Serving Love) - Kindle edition,Visit Amazon's K.A. Mitchell Page,Buy a Kindle,"[Kindle Store, Kindle eBooks, Literature & Fic..."
2,B0012W11D0,A1ZVW3VWAASU2Z,3.0,Just OK,Have to admit I expected more even though it i...,Hot Ticket (Serving Love) - Kindle edition,Visit Amazon's K.A. Mitchell Page,Buy a Kindle,"[Kindle Store, Kindle eBooks, Literature & Fic..."
3,B0012W11D0,A1XFKGZYK3N43L,5.0,When two world collides,This is the best to describe this book due to ...,Hot Ticket (Serving Love) - Kindle edition,Visit Amazon's K.A. Mitchell Page,Buy a Kindle,"[Kindle Store, Kindle eBooks, Literature & Fic..."
4,B0012W11D0,A13QTZ8CIMHHG4,5.0,Irresistable Read,Who knew community service could be so...sexy?...,Hot Ticket (Serving Love) - Kindle edition,Visit Amazon's K.A. Mitchell Page,Buy a Kindle,"[Kindle Store, Kindle eBooks, Literature & Fic..."


### Пункт 1: способы выделения названий (3 балла)
*Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon. Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?*

1. Использовать существительные / именные группы / именованные сущности из метаданных.
    * Суть: взять названия и описания товаров, извлечь названия и искать в отзывах
    * Данные: метаданные о товарах
    * Достоинства: дешево и сердито
    * Недостатки: очень грязный способ: при автоматическом выделении попадает много лишнего, трудно отсеять существительные / именные группы / сущности, которые называют именно сам товар; не факт, что в отзыве обязательно встретятся слова из названия/описания; если брать существительные, то длина названия ограничена одним словом
    * Возможное улучшение: добавить ручной отбор слов, которые попадают в потенциальные названия; взять названия (и м.б. описания) товаров, проанализировать, посмотреть на частотные существительные, составить список наименований
2. Использовать морфосинтаксические шаблоны (как в д/з 1).
    * Суть: взять названия, выделенные способом 1 (с улучшением), и составить шаблоны с ними
    * Данные: метаданные о товаров + собственная фантазия
    * Достоинства: длина наименования ограничена не одним словом, а тем количеством, которое допускают шаблоны
    * Недостатки: много мусора (в д/з 1 под шаблоны подходило много того, что не должно было бы); при написании правил невозможно учесть все реальные случаи
3. Использовать ключевые слова.
    * Суть: выделять ключевые слова из отзывов и искать пересечения с терминами, полученными из названий / описаний
    * Данные: метаданные о товарах
    * Достоинства: вычислительно проще, чем следующие два способа; в идеале алгоритмы будут находить слова, которые действительно важны для отзыва (в противовес обычному выделению существительных, которое даст кучу мусора)
    * Недостатки: очень строгий способ отбора слов (если берем пересечение, слов может вообще не остаться); алгоритмы выделения ключевых слов не идеальны
4. Использовать синонимы/гиперонимы/гипонимы.
    * Суть: расширить список слов из названия и описания товаров синонимичными словами и более общими названиями классов
    * Данные: метаданные о товарах + источники синонимов/гиперонимов/гипонимов (например, WordNet)
    * Достоинства: поможет избежать проблемы отсутствия слов из названия/описания в отзыве
    * Недостатки: потенциальное (еще большее) загрязнение данных -- попадание в них слов, которые не могут относиться к товару (т.к. в WordNet может быть огромное количество слов, если брать и синонимы, и гиперонимы, и гипонимы); невозможность искать синонимы для конструкций (только для отдельных слов)
5. Использовать векторные модели.
    * Суть: для выделенных в первом способе слов (из названия и описания) искать похожие слова внутри отзывов
    * Данные: метаданные + векторная модель
    * Достоинства: возможность бороться с отсутствием слов из названия/описания в отзыве + отсутствие мусора в данных (т.к. берем сразу только то, что гарантированно есть в отзывах)
    * Недостатки: непонятно, как определять "похожесть" слов. Просто самые похожие? Но это может быть что угодно, мало ли мусора в отзывах. Выбирать порог близости? Это не самая тривиальная задача и, пожалуй, больше в рамках информационного поиска. Наконец, как и в способе 3, нельзя посчитать близость, если в названии не 1 одно слово. (+ маленький размер корпуса, но можно просто взять датасет побольше)
6. Использовать готовые инструменты для извлечения именованных сущностей.
    * Суть: искать в отзывах именованные сущности и сопоставлять их с тем, что мы находим в названиях товаров (возможно, с помощью тех же инструментов
    * Данные: названия товаров
    * Достоинства: почти ничего не надо делать, все уже написано за нас
    * Недостатки: пожалуй, самый грязный способ, особенно на наших данных. Во-первых, в заголовках почти все слова с большой буквы, поэтому тот же spacy считает их все за имена собственные. Во-вторых, в отзывах никто обычно не пишет полное название, и даже не обязательно указывают бренд. Условно, если мы пишем отзыв на телефон Honor 10 xxx-xxx (код модели), мы напишем просто "телефон" или в крайнем случае "гаджет".

### Пункт 2: реализация одного из способов (2 балла)
*Реализуйте один из предложенных вами способов.*

Изначально я планировала реализовать способ с векторными моделями, но в процессе поняла, что комбинировать интереснее.

Первый шаг -- выделить из метаданных существительные, обозначающие товары. Попробуем начать с категорий: это закрытый класс, поэтому он чище, чем названия, в которые пытаются запихать всю возможную информацию (в т.ч. с сокращениями, что затрудняет частеречную разметку).
#### Категории + названия

In [26]:
categories = rev_df['category'].tolist()

In [27]:
all_cats = []
for item in categories:
    all_cats.extend(item)

In [28]:
cat_counter = Counter(all_cats)
print(len(cat_counter))
cat_counter.most_common(20)

51


[('Kindle Store', 485125),
 ('Kindle eBooks', 484161),
 ('Literature & Fiction', 222413),
 ('Romance', 93377),
 ('Science Fiction & Fantasy', 32885),
 ("Children's eBooks", 19671),
 ('Mystery, Thriller & Suspense', 19658),
 ('Religion & Spirituality', 17876),
 ('Health, Fitness & Dieting', 16579),
 ('Teen & Young Adult', 15407),
 ('Cookbooks, Food & Wine', 10021),
 ('Business & Money', 9903),
 ('Crafts, Hobbies & Home', 3266),
 ('Biographies & Memoirs', 2737),
 ('Humor & Entertainment', 2622),
 ('Self-Help', 2167),
 ('History', 2067),
 ('Parenting & Relationships', 2033),
 ('Reference', 1675),
 ('Education & Teaching', 1571)]

Кажется, нам повезло найти хороший однородный датасет (на самом деле никакого везения, я сменила четыре штуки...). Для дальнейшего анализа возьмем все, что в категории "Kindle eBooks."

In [29]:
sub_df = rev_df[pd.DataFrame(
    rev_df['category'].tolist()).isin(['Kindle eBooks']).any(1).values]
len(sub_df)

484161

Небольшая проблема: в категориях почти нет существительных, способных называть сам товар. Они описывают скорее жанр или тему. Из списка выше в качестве существительных мы скорее хотим видеть слова ebook и memoir, но не hobbies и relationships. Скорее всего, слова придется отбирать вручную. Но сначала разметим существительные с помощью spacy.

In [30]:
# если не работает, переустановить через командную строку
# минус два часа моей жизни
import spacy

In [31]:
nlp = spacy.load('en_core_web_sm')

Небольшая проблема -- spacy очень чувствителен к регистру и помечает все, что написано с большой буквы, как имя собственное. (NLTK, кстати, тоже этим страдает, я пробовала и его.) Придется привести все к нижнему регистру, что тоже не оптимально, потому что мы можем потерять какую-то важную информацию, но тут ничего не поделаешь: написание всего, кроме предлогов и союзов, с большой буквы, -- это специфика названий.

In [32]:
cats_lower = [cat.lower() for cat in list(set(all_cats))]

In [33]:
tagged_cats = []

for doc in tqdm(nlp.pipe(cats_lower, disable=['parser', 'ner']),
                total=len(cats_lower)):
    tagged_cats.append([(tok.text, tok.lemma_, tok.pos_) for tok in doc])

  0%|          | 0/51 [00:00<?, ?it/s]

Будем брать леммы слов, потому что категория часто называется во множественном числе, а сам товар скорее будет в единственном.

In [34]:
cat_nouns = []
for cat in tagged_cats:
    for pair in cat:
        if pair[-1] == 'NOUN' or pair[-1] == 'PROPN':
            cat_nouns.append(pair[1])

In [35]:
len(Counter(cat_nouns))

69

In [36]:
print([noun[0] for noun in Counter(cat_nouns).most_common()])

['kindle', 'page', 'minute', 'generation', 'science', 'paperwhite', 'fiction', 'hour', 'ebooks', 'art', 'entertainment', 'computer', 'technology', 'blog', 'romance', 'nonfiction', 'read', 'literature', 'health', 'fitness', 'dieting', 'law', 'biography', 'memoir', 'cookbooks', 'food', 'wine', 'touch', 'child', 'ebook', 'engineering', 'transportation', 'voyage', 'reference', 'mystery', 'thriller', 'suspense', 'business', 'money', 'dx', 'history', 'math', 'store', 'keyboard', 'teen', 'adult', 'politics', 'social', 'sport', 'outdoor', 'education', 'teaching', 'humor', 'comic_strip', 'manga', 'novel', 'parenting', 'relationship', 'religion', 'spirituality', 'travel', 'language', 'self', 'help', 'craft', 'hobby', 'home', 'photography', 'fantasy']


Отсюда нам подходят слова ebooks, manga, novel, ebook, cookbooks, thriller, biography, memoir (и то некоторые -- с натяжкой). Посмотрим, что бывает в названиях.

In [37]:
titles = [t.lower() for t in set(rev_df['title'].tolist())]

In [38]:
tagged_titles = []

for doc in tqdm(nlp.pipe(titles, disable=['parser', 'ner']),
                total=len(titles)):
    tagged_titles.append([(tok.text, tok.lemma_, tok.pos_) for tok in doc])

  0%|          | 0/31408 [00:00<?, ?it/s]

In [39]:
title_nouns = []
for title in tagged_titles:
    for pair in title:
        if pair[-1] == 'NOUN' or pair[-1] == 'PROPN':
            title_nouns.append(pair[1])

In [40]:
len(Counter(title_nouns))

13035

In [41]:
Counter(title_nouns).most_common(20)

[('kindle', 23844),
 ('edition', 23755),
 ('book', 12237),
 ('ebook', 4404),
 ('series', 3809),
 ('romance', 1698),
 ('story', 1486),
 ('recipe', 1274),
 ('love', 1241),
 ('guide', 854),
 ('life', 757),
 ('mystery', 751),
 ('child', 701),
 ('kid', 668),
 ('weight', 634),
 ('diet', 590),
 ('novel', 542),
 ('christmas', 539),
 ('heart', 523),
 ('trilogy', 513)]

Ого, получилось довольно осмысленно! Я предполагала, что существительные из названий окажутся бесполезными для нашей задачи, потому что в отзыве человек скорее напишет "эта книга / серия / роман", а не само название, но, очевидно, названия электронных книг на Амазоне не ограничиваются собственно заглавием и именем автора.

Посмотрим на 50 самых частотных существительных и вручную выберем те из них, которые могут относиться к товару.

In [42]:
print([noun[0] for noun in Counter(title_nouns).most_common(100)])

['kindle', 'edition', 'book', 'ebook', 'series', 'romance', 'story', 'recipe', 'love', 'guide', 'life', 'mystery', 'child', 'kid', 'weight', 'diet', 'novel', 'christmas', 'heart', 'trilogy', 'part', 'loss', 'man', 'collection', 'day', 'novella', 'time', 'secret', 'billionaire', 'family', 'beginner', 'night', 'picture', 'tale', 'girl', 'animal', 'adventure', 'publishing', 'woman', 'world', 'bride', 'wolf', 'fire', 'volume', 'self', 'blood', 'home', 'fact', 'way', 'cookbook', 'health', '.', 'thriller', 'stress', 'set', 'age', 'food', 'oil', 'angel', 'dog', 'money', 'star', 'murder', 'dream', 'moon', 'baby', 'saga', 'step', 'boy', 'fun', 'order', 'friend', 'game', 'tip', 'chronicle', 'box', 'regency', 'club', 'alpha', 'desire', 'shadow', 'war', 'cure', 'menage', 'dragon', 'mail', 'body', 'cowboy', 'suspense', 'soul', 'king', 'cat', 'vampire', 'adult', 'chance', 'anxiety', 'lover', 'paleo', 'rock', 'power']


Интересная тенденция: чем выше слово в списке, тем больше оно нам подходит; чем ниже, тем больше вероятность, что это часть темы или названия. Теперь выберем нужные нам слова, не забыв про те, которые мы выделили из категорий.

In [43]:
noun_set = ['ebooks', 'manga', 'novel', 'ebook',
            'thriller', 'biography', 'memoir'] + \
           ['edition', 'book', 'series', 'story',
            'guide', 'novel', 'trilogy', 'collection',
            'novella', 'tale', 'volume', 'cookbook', 'saga']

#### Отзывы
Теперь посмотрим на то, что содержится в отзывах.

In [44]:
texts = rev_df['reviewText'].tolist()

ЧАСТЬ С РАЗМЕТКОЙ -- НЕ ЗАПУСКАТЬ ИНАЧЕ СМЕРТЬ

In [ ]:
text_nouns = []
lem_texts = []

for doc in tqdm(nlp.pipe(texts, disable=['parser', 'ner']),
                total=len(texts)):
    lem_text = []

    for tok in doc:
        lemma = tok.lemma_
        pos = tok.pos_
        if pos != 'PUNCT':
            lem_text.append(lemma)
        if pos == 'NOUN' or pos == 'PROPN':
            text_nouns.append(lemma)

    lem_texts.append(lem_text)

In [ ]:
with open('tagged_texts.pickle', 'wb') as f:
    pickle.dump((Counter(text_nouns).most_common(100),
                 lem_texts), f)

ЗАПУСКАТЬ ОТСЮДА

In [45]:
with open('tagged_texts.pickle', 'rb') as f:
    noun_counter, lem_texts = pickle.load(f)

In [46]:
print([w[0] for w in noun_counter])

['book', 'story', 'character', 'series', 'author', 'time', 'love', 'life', 'way', 'read', 'thing', 'one', 'romance', 'man', 'lot', 'friend', 'end', 'part', 'people', 'year', 'review', 'woman', 'relationship', 'reader', 'family', 'world', 'bit', 'page', 'novel', 'plot', 'sex', 'day', 'scene', 'heart', 'star', 'job', 'writing', 'girl', 'ending', 'action', 'child', 'work', 'line', 'guy', 'word', 'point', 'place', 'couple', 'fact', 'idea', 'twist', 'fun', 'novella', 'recipe', 'person', 'feeling', 'fan', 'brother', 'mystery', 'problem', 'other', 'issue', 'past', 'beginning', 'moment', 'hero', 'heroine', 'information', 'mind', 'night', 'chapter', 'father', 'copy', 'mother', 'tale', 'side', 'emotion', 'reason', 'sister', 'kid', 'reading', 'eye', 'Ms.', 'boy', 'writer', 'detail', 'storyline', 'chance', 'style', 'situation', 'money', 'kind', 'type', 'sense', 'home', 'self', 'parent', 'school', 'town', 'head']


Тут все вполне логично с точки зрения содержания. Кажется, часть этих слов тоже имеет смысл включить в итоговый набор.

In [47]:
noun_set += ['book', 'story', 'series', 'novel', 'novella', 'tale']
core = set(noun_set)

In [48]:
len(core)

19

In [49]:
print(core)

{'tale', 'ebook', 'collection', 'trilogy', 'memoir', 'book', 'volume', 'ebooks', 'manga', 'series', 'edition', 'thriller', 'novella', 'biography', 'cookbook', 'saga', 'guide', 'story', 'novel'}


#### Векторная модель
Перейдем к обучению векторной модели на нашем корпусе. У нас уже есть распаршенные с помощью spacy тексты, осталось привести их к виду текстов (а не списков кортежей с тегами) и передать модели на обучение.

НЕ ЗАПУСКАТЬ ОБУЧЕНИЕ МОДЕЛИ

In [ ]:
model = gensim.models.Word2Vec(lem_texts, window=3,
                               min_count=2, seed=seed)
model.wv.save('word2vec.wordvectors')

ЗАПУСКАТЬ ОТСЮДА

In [50]:
model_wv = KeyedVectors.load('word2vec.wordvectors', mmap='r')

In [51]:
model_wv.most_similar('book', topn=10)

[('novel', 0.7926753163337708),
 ('novella', 0.7717196941375732),
 ('story', 0.7468701004981995),
 ('series', 0.74058598279953),
 ('trilogy', 0.6998007297515869),
 ('installment', 0.6991181969642639),
 ('volume', 0.6868460774421692),
 ('ebook', 0.6847478747367859),
 ('cookbook', 0.6442406177520752),
 ('episode', 0.6115556359291077)]

In [52]:
model_wv.most_similar('memoir', topn=10)

[('biography', 0.7878738045692444),
 ('poetry', 0.7064180374145508),
 ('essay', 0.6857961416244507),
 ('diary', 0.6478056311607361),
 ('nonfiction', 0.6450907588005066),
 ('poet', 0.6251628398895264),
 ('parable', 0.6183352470397949),
 ('screenplay', 0.6136636137962341),
 ('poem', 0.605504035949707),
 ('literature', 0.5864816904067993)]

Кажется, модель неплохо обучилась. Я пробовала использовать готовые эмбеддинги (обученные на твиттере и википедии), но они выдают результаты, не специфичные для нашего корпуса (а для нас это скорее плюс, чем минус).

Итак, мы получили набор существительных, которые могут называть товары в нашем датасете, и векторную модель, обученную на отзывах.

#### Выделение сущностей из отзывов
Попробуем несколько вариантов:
* Брать пересечение слов из отзыва со словами core
* Выделять ключевые слова и конструкции и выбирать те, в которых есть слова из core
* Выделять названия товаров из отзыва как самые близкие к словам из core

#### Пересечение

In [53]:
common_words = []
for review in tqdm(lem_texts):
    common_words.append(set(review).intersection(core))

  0%|          | 0/485125 [00:00<?, ?it/s]

In [54]:
for i in range(10):
    if len(lem_texts[i]) < 100:
        print(common_words[i],
              '\n',
              texts[i],
              '\n')

{'novella'} 
 Light, short novella, nothing too deep - Cade and Elliot meet doing trash recycling for community time. One is a raunchy waiter, the other younger and more conservative, and they connect and go on from there. It's actually quite forgettable, but not at all offensive... I rate it B+ but not necessarily a reread and certainly not essential to own (unlike the excellent Jez Morrow's Force of Law - against which all other m/m novellas are judged and found wanting...) 

{'story'} 
 Have to admit I expected more even though it is a short story. Just fell a little flat. Not my favourite. Sorry 

{'book'} 
 This is the best to describe this book due to the main characters are from opposite side of the track and comes from two different worlds but in serving community service they meet and find that are so much alike and have so much in common. In the difference they fall in love and build the perfect relationship and learn how to love and care for each other and over look the diff

Большая проблема: если человек пишем в отзыве что-то типа "мне нравятся книги, которые...". Такие случаи выловить нашим методом невозможно, и кажется, что и векторная модель тут не особенно поможет. Тем не менее подход с моделью более гибкий, т.к. позволяет выбирать слова, специфичные для конкретного отзыва.

#### Похожие слова
Этот способ очень долгий из-за вычисления близости с каждым словом (даже если считать близость только к слову "book"), поэтому я сделала подсчет только для части отзывов, чтобы проверить, что получается.

In [55]:
stops = set(stopwords.words('english') + ['I'])

In [56]:
closest_words = []
for review in tqdm(lem_texts[:10000]):
    sim_dict = {}
    # для каждого уникального слова вычисляем близость
    for word in list(set(review) - stops):
        if word in model_wv:
            sim_dict[word] = model_wv.similarity('book', word)
    # берем 5 ближайших слов
    closest_words.append(nlargest(5, sim_dict, key=sim_dict.get))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [57]:
for i in range(10):
    if len(lem_texts[i]) < 100:
        print(closest_words[i],
              '\n',
              texts[i],
              '\n')

['novella', 'one', 'time', 'short', 'actually'] 
 Light, short novella, nothing too deep - Cade and Elliot meet doing trash recycling for community time. One is a raunchy waiter, the other younger and more conservative, and they connect and go on from there. It's actually quite forgettable, but not at all offensive... I rate it B+ but not necessarily a reread and certainly not essential to own (unlike the excellent Jez Morrow's Force of Law - against which all other m/m novellas are judged and found wanting...) 

['story', 'short', 'though', 'expect', 'even'] 
 Have to admit I expected more even though it is a short story. Just fell a little flat. Not my favourite. Sorry 

['book', 'character', 'world', 'relationship', 'different'] 
 This is the best to describe this book due to the main characters are from opposite side of the track and comes from two different worlds but in serving community service they meet and find that are so much alike and have so much in common. In the differen

Здесь получается гораздо больше мусора: если не убирать стоп-слова, то они составляют около половины всей выдачи, но и при их удалении выдача не намного чище, поскольку мы требуем всегда выдавать топ-5 ближайших слов. Эта проблема опять сводится к выбору порога и, на мой взгляд, выходит за рамки этого домашнего задания.

#### Ключевые слова
В д/з 1 лучшим алгоритмом выделения ключевых слов оказался TF-IDF, но, как известно, этот метод не предназначен для работы с корпусами однородных текстов, а именно с таким корпусом мы сейчас имеем дело. На втором месте по качеству оказался TextRank, так что используем его.

Сначала выделим ключевые слова, а затем проверим, какие из них входят в core. Это тоже работает очень долго, поэтому проверим на части корпуса...

In [58]:
joined_lemmas = [' '.join(text) for text in tqdm(lem_texts)]

  0%|          | 0/485125 [00:00<?, ?it/s]

In [59]:
textrank_kws = []

for text in tqdm(joined_lemmas[:1000]):
    kw_list = keywords.keywords(text, language='english',
                                ratio=0.5,
                                additional_stopwords=stops).split('\n')
    textrank_kws.append(set(kw_list).intersection(core))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [60]:
textrank_kws[0:10]

[{'book'}, set(), set(), set(), {'book'}, set(), set(), set(), set(), set()]

Отличный способ, я победила эту домашку))

Из трех испробованных методов наиболее удачным, как ни странно, оказался самый простой: использование core для выделения упоминаний из отзывов. В дальнейшем можно расширять core разными способами: вручную, просматривая словари синонимов; используя WordNet; повторяя наш алгоритм на новых данных.

### Пункт 3: n-граммы с полученными сущностями (1 балл)
Поскольку все сущности у нас однословные, то под шаблон "NE + левый сосед / NE + правый сосед" подходят только биграммы.

In [61]:
lem_texts_lower = [t.lower().split() for t in tqdm(joined_lemmas)]

  0%|          | 0/485125 [00:00<?, ?it/s]

In [62]:
bigrams = Counter()

for text in tqdm(lem_texts_lower):

    # отсекаем тексты, где нет ничего подходящего
    if not set(text).intersection(core):
        continue

    else:
        bigrams.update(list(ngrams(text, 2)))

  0%|          | 0/485125 [00:00<?, ?it/s]

In [63]:
core_bigrams = []
for bigr in tqdm(bigrams.most_common()):
    if set(bigr[0]).intersection(core):
        core_bigrams.append(bigr)

  0%|          | 0/4230116 [00:00<?, ?it/s]

### Пункт 4: ранжирование n-грамм
#### PMI

In [64]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [65]:
bigram_finder = BigramCollocationFinder.from_documents(lem_texts_lower)

In [66]:
# корпус большой, так что и фильтр побольше
# кстати, он очень влияет, на 5 было много мусора
bigram_finder.apply_freq_filter(10)

In [67]:
bigrams_pmi = bigram_finder.score_ngrams(bigram_measures.pmi)

In [68]:
def arrange_bigrams(bigrams_with_scores, core_bigrams):
    bigrams_no_scores = [bigr[0] for bigr in bigrams_with_scores]
    core_bigrams_no_freq = set([bigr[0] for bigr in core_bigrams])
    arranged_bigrams = [bigr for bigr in tqdm(bigrams_no_scores)
                        if bigr in core_bigrams_no_freq]
    return arranged_bigrams

In [69]:
arranged_by_pmi = arrange_bigrams(bigrams_pmi, core_bigrams)

  0%|          | 0/320707 [00:00<?, ?it/s]

In [70]:
arranged_by_pmi[0:20]

[('fairytail', 'saga'),
 ('omnibus', 'edition'),
 ('wardstone', 'trilogy'),
 ('artists', 'trilogy'),
 ('bloodstone', 'saga'),
 ('riverside', 'trilogy'),
 ('seen', 'trilogy'),
 ('lodestone', 'trilogy'),
 ('newsflesh', 'trilogy'),
 ('techno', 'thriller'),
 ('colter', 'saga'),
 ('firebird', 'trilogy'),
 ('enslaved', 'trilogy'),
 ('255', 'guide'),
 ('cautionary', 'tale'),
 ('details', 'ebook'),
 ('psychological', 'thriller'),
 ('revise', 'edition'),
 ('fairy', 'tale'),
 ('beginners', 'guide')]

С одной стороны, тут есть полезные биграммы с именами собственными; с другой, непонятно, насколько они отражают именно наименования тех товаров, к которым относится отзыв (т.е. я скорее представляю себе отзыв типа "эта книга фигня, а вот firebird trilogy..." -- но это моя интуиция, и неясно, обоснованная ли). Ну и, конечно, заметна известная склонность PMI переоценивать редкие слова.

#### T-score

In [71]:
bigrams_t = bigram_finder.score_ngrams(bigram_measures.student_t)

In [72]:
arranged_by_t = arrange_bigrams(bigrams_t, core_bigrams)

  0%|          | 0/320707 [00:00<?, ?it/s]

In [73]:
arranged_by_t[0:20]

[('this', 'book'),
 ('the', 'story'),
 ('the', 'book'),
 ('this', 'story'),
 ('this', 'series'),
 ('book', 'be'),
 ('short', 'story'),
 ('the', 'series'),
 ('book', 'in'),
 ('next', 'book'),
 ('book', 'i'),
 ('story', 'be'),
 ('first', 'book'),
 ('story', 'line'),
 ('love', 'story'),
 ('great', 'book'),
 ('great', 'story'),
 ('story', 'that'),
 ("'s", 'story'),
 ('other', 'book')]

Выдача довольно скучная (помним о склонность t-score переоценивать частотные слова), но, скорее всего, отражающая реальность: все топ-5 биграмм представляются как те, что могут относиться к товарам в отзывах.

#### Chi-square

In [74]:
bigrams_chi = bigram_finder.score_ngrams(bigram_measures.chi_sq)

In [75]:
arranged_by_chi = arrange_bigrams(bigrams_chi, core_bigrams)

  0%|          | 0/320707 [00:00<?, ?it/s]

In [76]:
arranged_by_chi[0:20]

[('this', 'book'),
 ('fairy', 'tale'),
 ('story', 'line'),
 ('short', 'story'),
 ('the', 'story'),
 ('this', 'series'),
 ('debut', 'novel'),
 ('length', 'novel'),
 ('next', 'book'),
 ('psychological', 'thriller'),
 ('this', 'story'),
 ('omnibus', 'edition'),
 ('kindle', 'edition'),
 ('artists', 'trilogy'),
 ('first', 'book'),
 ('romance', 'novel'),
 ('cautionary', 'tale'),
 ('fairytail', 'saga'),
 ('second', 'book'),
 ('the', 'book')]

Чем-то похоже на то, что у t-score: здесь есть всякие this book, this series, this story, то есть как раз то, что мы хотим видеть в выдаче. Под конец появляются сущности, также встречающиеся в топе PMI. Как ни странно, кажется, что t-score справляется лучше: хоть он и переоценивает частотные слова, его выдача лучше соотносится с наименованиями товаров в отзывах (по крайней мере, в моем воображении).

Попробуем взять топ-100 биграмм в рейтингах t-score и chi-square и их пересечение.

In [77]:
t_and_chi = arranged_by_t[0:100] + arranged_by_chi[0:100]

In [78]:
top_t_and_chi = []
for bigram in Counter(t_and_chi).most_common():
    if bigram[-1] < 2:
        break
    else:
        bigram_str = ''
        for word in bigram[0]:
            bigram_str += ' '
            bigram_str += word
        if not re.search(r'[^a-z0-9\s]', bigram_str):
            top_t_and_chi.append(bigram_str.strip())

In [79]:
len(top_t_and_chi)

49

In [80]:
print(top_t_and_chi)

['this book', 'the story', 'the book', 'this story', 'this series', 'book be', 'short story', 'the series', 'book in', 'next book', 'book i', 'story be', 'first book', 'story line', 'love story', 'great book', 'great story', 'story that', 'other book', 'good book', 'second book', 'series i', 'book by', 'story about', 'series and', 'good story', 'these book', 'this novella', 'this novel', 'book 2', 'book down', 'book 1', 'romance novel', 'previous book', 'third book', 'tale of', 'collection of', 'book 3', 'whole series', 'fairy tale', 'wonderful story', 'new series', 'length novel', 'entire series', 'entire book', 'cute story', 'story itself', 'this ebook', 'story flow']


Получается довольно много правильного и полезного. В принципе, можно использовать обе метрики, но я боюсь сделать логическую ошибку, пытаясь их объединить. Можно просто для каждого товара взять топ-n и их пересечение, но хочется учитывать не просто факт вхождения биграммы в топ, но и ее позицию в нем, и вот это я уже плохо себе представляю, как реализовывать. Наверное, можно нормализовать значения каждого типа метрик, сложить и ранжировать, а затем взять топ-n, но я не успею попробовать и понять, не ерунду ли предложила :(

#### Группировка биграмм по NE (1 балл)
В итоге я выбрала метрику t-score, потому что она высоко ранжирует хоть и базовые, зато с большой вероятностью подходящие сущности. 

In [81]:
core_list = list(core)
core_collocations = {}

for word in tqdm(core_list):
    core_collocations[word] = []
    # будем записывать в том порядке,
    # которые предложил t-score
    for bigram in arranged_by_t:
        if word in bigram:
            bigram_str = ' '.join(str(w) for w in bigram)
            if not re.search(r'[^a-z0-9\s]', bigram_str):
                core_collocations[word].append(bigram_str)

  0%|          | 0/19 [00:00<?, ?it/s]

In [82]:
show_results = ['book', 'series', 'story', 'tale', 'guide']
for word in show_results:
    print(word, '\n---')
    print('\n'.join(core_collocations[word][0:5]), '\n')

book 
---
this book
the book
book be
book in
next book 

series 
---
this series
the series
series i
series and
a series 

story 
---
the story
this story
short story
story be
story line 

tale 
---
tale of
fairy tale
this tale
tale that
the tale 

guide 
---
guide to
this guide
guide for
great guide
guide on 



Получилось довольно неплохо! Интересно, каких результатов можно было бы добиться, объединив метрики предложенным выше способом (или каким-то другим, принятым среди компьютерных лингвистов, которые уже закончили бакалавриат...).

Из минусов: кажется, мы не очень-то выделяем имена собственные. Две причины, почему это не критично: 1) они на самом деле есть в топе t-score, просто не в топ-5, а где-то в топ-15-20; 2) специфика датасета: имена собственные здесь вряд ли будут отражать собственно тот товар, на который пишется отзыв. Больше похоже, что они будут отсылать к каким-то сущностям из самой книги, или к другим книгам серии, или книгам того же автора/жанра, и т.д.

#### Бонус: объединение синонимов
Поскольку у меня не остается времени на реализацию, то на бонус я не претендую, просто выскажу свою идею. Для объединения синонимичных употреблений можно попробовать кластеризацию, основанную на векторном представлении слов (у нас как раз есть обученная на отзывах модель). Например, создать векторные представления товаров на основе их названий и кластеризовать их (правда, кажется, что название самого произведения нас может сбить, оно может быть совершенно произвольным; но для случая с часами этот способ должен подойти).

И еще про то, как можно улучшить качество: кажется, имеет смысл использовать не леммы, а леммы + части речи (склеив их через нижнее подчеркивание). Это помогло бы бороться со случаями типа read (сущ.: it's a great read) и потенциально улучшило бы работу с векторной моделью. Еще можно доработать алгоритмы, учитывающие ключевые слова, и выделять не только уни-, но и n-граммы, а затем брать те, в которые входит одно из core-слов.